# CTE

Gunakan rexon_metals.db!

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#import packages
import os
import pandas as pd
import sqlite3 as sq

In [3]:
# Kemudian kita definisikan path folder dimana file database kita disimpan
path = "gdrive/MyDrive/MBKM/Week 4 - SQL/"

In [4]:
# untuk mengecek ada file apa saja didalam path yang digunakan
os.listdir(path)

['weather_stations.db',
 'transportation_sales.db',
 'Meet 8 SQL for Data Analysis Part II.ipynb',
 'Mainroom_Meet_9.ipynb',
 'rexon_metals.db',
 'Exercise_7_Fariz_Lintang Bima Sakti S.ipynb',
 'Exercise_8_Fariz_Lintang Bima Sakti S.ipynb']

In [5]:
# Kemudian kembali membuat connection ke database yang udah ada
conn = sq.connect(path + 'rexon_metals.db')

# Membuat cursor object untuk memanggil SQL statement
cur = conn.cursor()

In [6]:
# Kemudian kita dapat execute SQL statement. Kali ini kita akan mengecek terdapat tabel apa saja di rexon_metals.db
cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

[('PRODUCT',), ('sqlite_sequence',), ('CUSTOMER',), ('CUSTOMER_ORDER',), ('stocks',)]


### Membaca semua tabel pada database

In [7]:
pd.read_sql_query("""
                        SELECT * FROM PRODUCT
                        """,conn)

,PRODUCT_ID,DESCRIPTION,PRICE
0,1,Copper,7.51
1,2,Aluminum,2.58
2,3,Silver,15.00
3,4,Steel,12.31
4,5,Bronze,4.00
5,6,Duralumin,7.60
6,7,Solder,14.16
7,8,Stellite,13.31
8,9,Brass,4.75


In [8]:
pd.read_sql_query("""
                        SELECT * FROM CUSTOMER
                        """,conn)

,CUSTOMER_ID,NAME,REGION,STREET_ADDRESS,CITY,STATE,ZIP
0,1,LITE Industrial,Southwest,729 Ravine Way,Irving,TX,75014
1,2,Rex Tooling Inc,Southwest,6129 Collie Blvd,Dallas,TX,75201
2,3,Re-Barre Construction,Southwest,9043 Windy Dr,Irving,TX,75032
3,4,Prairie Construction,Southwest,264 Long Rd,Moore,OK,62104
4,5,Marsh Lane Metal Works,Southeast,9143 Marsh Ln,Avondale,LA,79782


In [9]:
pd.read_sql_query("""
                        SELECT * FROM CUSTOMER_ORDER
                        """,conn)

,ORDER_ID,ORDER_DATE,SHIP_DATE,CUSTOMER_ID,PRODUCT_ID,ORDER_QTY,SHIPPED
0,1,2015-05-15,2015-05-18,1,1,450,false
1,2,2015-05-18,2015-05-21,3,2,600,false
2,3,2015-05-20,2015-05-23,3,5,300,false
3,4,2015-05-18,2015-05-22,5,4,375,false
4,5,2015-05-17,2015-05-20,3,2,500,false


In [10]:
pd.read_sql_query("""
                        SELECT * FROM STOCKS
                        """,conn)

,date,trans,symbol,qty,price
0,2006-01-05,BUY,RHAT,100.0,35.14


#### Total hasil penjualan setiap produk

In [11]:
# Step 1: mengetahui total kuantiti terjual
totalkuantiti = pd.read_sql_query("""
                                SELECT PRODUCT_ID,SUM(ORDER_QTY) AS TOTAL_KUANTITAS
                                FROM CUSTOMER_ORDER
                                GROUP BY PRODUCT_ID
""",conn)
totalkuantiti

,PRODUCT_ID,TOTAL_KUANTITAS
0,1,450
1,2,1100
2,4,375
3,5,300


In [12]:
# Step 2: mengalikan kuantiti terjual dengan harga jual
res = pd.read_sql_query("""
                                SELECT CO.PRODUCT_ID, (P.PRICE * CO.ORDER_QTY) AS TOTAL_PENDAPATAN
                                FROM CUSTOMER_ORDER CO, PRODUCT P
                                WHERE P.PRODUCT_ID = CO.PRODUCT_ID
                                GROUP BY CO.PRODUCT_ID
""",conn)
res

,PRODUCT_ID,TOTAL_PENDAPATAN
0,1,3379.50
1,2,1548.00
2,4,4616.25
3,5,1200.00
